In [14]:
# ============================ interactive_dashboard.py ============================
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

sns.set(style="whitegrid")
st.set_page_config(page_title="DataWave Churn Dashboard", layout="wide")

# ============================ 1️⃣ Upload Data ============================
st.title("📊 DataWave Music Churn Dashboard")
file = st.file_uploader("Upload CSV", type=["csv"])

if file:
    df = pd.read_csv(file)

    # ============================ 2️⃣ Data Cleaning ============================
    df = df.drop_duplicates(subset=['user_id']).dropna(subset=['user_id'])
    df = df.dropna(subset=['gender'])
    df['gender'] = df['gender'].str.lower().str.strip().map({'f':'Female','female':'Female','m':'Male','male':'Male','other':'Other'})
    
    df['country'] = df['country'].str.lower().str.strip().str.replace('.', '', regex=False)
    country_map = {'uk':'United Kingdom','united kingdom':'United Kingdom','usa':'United States','ind':'India'}
    df['country'] = df['country'].map(country_map).fillna(df['country']).str.title()
    country_to_region = {'United States':'North America','United Kingdom':'Europe','India':'Asia'}
    df['region'] = df['country'].map(country_to_region).fillna('Other')
    
    developed_countries = ['United States','United Kingdom']
    df['market_type'] = df['country'].apply(lambda x: 'Developed' if x in developed_countries else 'Emerging')
    
    df['subscription_type'] = df['subscription_type'].str.lower().str.strip().map({
        'studnt':'Student','fam':'Family','premum':'Premium'
    }).fillna(df['subscription_type']).str.title()
    
    df = df.dropna(subset=['satisfaction_score'])
    
    def clean_skip_rate(x):
        if pd.isna(x): return np.nan
        x = str(x).strip().lower()
        if "%" in x: x = x.replace("%",""); return pd.to_numeric(x, errors="coerce")/100
        num = pd.to_numeric(x, errors="coerce")
        if pd.notna(num): return num
        words = {"zero":0,"one":1,"two":2,"three":3,"four":4,"five":5,"six":6,"seven":7,"eight":8,"nine":9,"ten":10}
        return words.get(x, np.nan)
    
    df["skip_rate"] = df["skip_rate"].apply(clean_skip_rate)
    df["skip_rate"] = df["skip_rate"].apply(lambda x: x/100 if pd.notna(x) and x>1 else x)
    
    df['churned'] = df['churned'].astype(str).str.capitalize().str.strip().map({'1':'Yes','0':'No'}).map({'No':0,'Yes':1})
    
    def clean_monthly_fee(x):
        if pd.isna(x): return np.nan
        x = str(x).strip().lower().replace("usd","").replace("$","").strip()
        num = pd.to_numeric(x, errors='coerce')
        if pd.notna(num): return num
        if x in ["free","none","no fee"]: return 0.0
        return np.nan
    df['monthly_fee_clean'] = df['monthly_fee'].apply(clean_monthly_fee)
    
    df['join_date'] = pd.to_datetime(df['join_date'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['join_date'])
    today = pd.to_datetime('today')
    df['tenure_months'] = (today.year - df['join_date'].dt.year)*12 + (today.month - df['join_date'].dt.month)
    
    def age_group(age):
        if pd.isna(age): return 'Unknown'
        age = int(age)
        if age <= 18: return '13-18'
        elif age <= 24: return '19-24'
        elif age <= 34: return '25-34'
        elif age <= 44: return '35-44'
        elif age <= 54: return '45-54'
        elif age <= 64: return '55-64'
        else: return '65+'
    df['age_group'] = df['age'].apply(age_group)
    
    # ============================ 3️⃣ Sidebar Filters ============================
    st.sidebar.header("Filter Data")
    region_filter = st.sidebar.multiselect("Region", options=df['region'].unique(), default=df['region'].unique())
    subscription_filter = st.sidebar.multiselect("Subscription Type", options=df['subscription_type'].unique(), default=df['subscription_type'].unique())
    age_filter = st.sidebar.multiselect("Age Group", options=df['age_group'].unique(), default=df['age_group'].unique())
    
    filtered_df = df[df['region'].isin(region_filter) & 
                     df['subscription_type'].isin(subscription_filter) &
                     df['age_group'].isin(age_filter)]
    
    # ============================ 4️⃣ Display Metrics ============================
    st.subheader("Overall Metrics")
    col1, col2, col3, col4 = st.columns(4)
    col1.metric("Overall Churn", f"{filtered_df['churned'].mean()*100:.2f}%")
    col2.metric("Retention Rate", f"{100 - filtered_df['churned'].mean()*100:.2f}%")
    col3.metric("Avg Weekly Listening", f"{filtered_df['avg_listening_hours_per_week'].mean():.2f} hrs")
    col4.metric("Avg Satisfaction", f"{filtered_df['satisfaction_score'].mean():.2f}/5")
    
    # ============================ 5️⃣ Churn Analysis ============================
    st.subheader("Churn Rate by Subscription Type")
    sub_churn = filtered_df.groupby('subscription_type')['churned'].mean()*100
    fig, ax = plt.subplots(figsize=(6,4))
    sns.barplot(x=sub_churn.index, y=sub_churn.values, palette='Set2', ax=ax)
    ax.set_ylabel("Churn Rate (%)")
    st.pyplot(fig)
    
    st.subheader("Churn Rate by Region")
    region_churn = filtered_df.groupby('region')['churned'].mean()*100
    fig, ax = plt.subplots(figsize=(6,4))
    sns.barplot(x=region_churn.index, y=region_churn.values, palette='Set1', ax=ax)
    ax.set_ylabel("Churn Rate (%)")
    st.pyplot(fig)
    
    st.subheader("Churn Rate by Age Group")
    age_churn = filtered_df.groupby('age_group')['churned'].mean()*100
    fig, ax = plt.subplots(figsize=(6,4))
    sns.barplot(x=age_churn.index, y=age_churn.values, palette='coolwarm', ax=ax)
    ax.set_ylabel("Churn Rate (%)")
    st.pyplot(fig)
    
    # ============================ 6️⃣ Risk Score ============================
    st.subheader("Churn Risk Score (Logistic Regression)")
    numeric_cols = ['total_songs_played','satisfaction_score','monthly_fee_clean','age','avg_listening_hours_per_week','skip_rate']
    X = filtered_df[numeric_cols].fillna(0)
    y = filtered_df['churned']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    logreg = LogisticRegression(max_iter=1000)
    logreg.fit(X_scaled, y)
    filtered_df['churn_risk_score'] = logreg.predict_proba(X_scaled)[:,1]
    
    risk_bins = pd.qcut(filtered_df['churn_risk_score'], q=5, labels=False)
    risk_summary = filtered_df.groupby(risk_bins)['churned'].agg(['count','mean']).rename(columns={'mean':'actual_churn_rate'})
    
    fig, ax = plt.subplots(figsize=(6,4))
    sns.barplot(x=risk_summary.index, y=risk_summary['actual_churn_rate']*100, palette='Reds', ax=ax)
    ax.set_ylabel("Churn Rate (%)")
    ax.set_xlabel("Risk Score Quintile")
    ax.set_title("Actual Churn by Predicted Risk Score")
    st.pyplot(fig)
    
    # ============================ 7️⃣ Annual & Quarterly Cohorts ============================
    st.subheader("Annual Cohort Churn Rate")
    filtered_df['join_year'] = filtered_df['join_date'].dt.year
    annual_cohorts = filtered_df.groupby('join_year').agg(users=('user_id','count'), churned=('churned','sum'))
    annual_cohorts['churn_rate'] = annual_cohorts['churned']/annual_cohorts['users']*100
    st.bar_chart(annual_cohorts['churn_rate'])
    
    st.subheader("Quarterly Cohort Churn Rate")
    filtered_df['join_year_quarter'] = filtered_df['join_date'].dt.to_period('Q')
    quarterly_cohorts = filtered_df.groupby('join_year_quarter').agg(users=('user_id','count'), churned=('churned','sum'))
    quarterly_cohorts['churn_rate'] = quarterly_cohorts['churned']/quarterly_cohorts['users']*100
    st.line_chart(quarterly_cohorts['churn_rate'])
    
    # ============================ 8️⃣ Strategic Recommendations ============================
    st.subheader("Strategic Recommendations")
    st.write("""
    1. Focus retention campaigns on high-risk quintiles identified by the risk score model.  
    2. Free-tier high engagement users could be converted to paid plans with targeted offers.  
    3. Newly joined users (6-12 months) require onboarding engagement.  
    4. Address churn drivers in emerging markets; consider region-specific offers or pricing adjustments.  
    5. Age groups 25-34 and 35-44 show higher churn; investigate content and subscription satisfaction.  
    6. Subscription type impacts churn: Student & Family plans require targeted incentives to improve retention.
    """)


2025-11-28 09:15:42.310 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-28 09:15:42.311 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-28 09:15:42.848 
  command:

    streamlit run c:\Users\bista\OneDrive\Desktop\IOA_Project\.venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-28 09:15:42.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-28 09:15:42.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-28 09:15:42.851 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-28 09:15:42.852 Thread 'MainThread': missing ScriptRunContext! This warning